# Some experiments with OpenCV

In [ ]:
import cv2
import io
import numpy as np
import urllib.request
import pandas as pd

from PIL import Image as PImage
import matplotlib.pyplot as plt
import os

In [ ]:
# helper functions to convert between PIL and OpenCV image types

def tocv(pil):
  return np.array(pil)

def topil(cv):
  return PImage.fromarray(cv)

## Open an Image

In [ ]:
# open image from url
img = PImage.open("random_subset/plant_nadeshiko_1.jpg")
# img = PImage.open("random_subset/plant_kiri_113.jpg")

# this makes the largest edge of the image be 480
img.thumbnail((480, 480))
display(img)

## Threshold

Turn color/gray image into black and white

Doc: https://docs.opencv.org/4.x/db/d8e/tutorial_threshold.html

"In OpenCV, finding contours is like finding white object from black background. So remember, object to be found should be white and background should be black." https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html


In [ ]:
img_cv = tocv(img)
#and display pixel intensity distribution
plt.hist(img_cv.ravel(), bins=256, range=(0, 255))
plt.title("Pixel Intensity Distribution")
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.show()


#convert all images to grayscale if not greyscale
if len(img_cv.shape) == 3 and img_cv.shape[2] == 3:  # If the image has 3 channels (RGB)
    img_cv_grey = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)
else:  # If the image is already grayscale
    img_cv_grey = img_cv
    img_cv_grey = cv2.equalizeHist(img_cv_grey) #images that are already greyscale have low contrast
    # so we equalize the intesity of each pixel



#threshold image to binary
max_value = 255
threshold_val = 128
_, img_cv_grey = cv2.threshold(img_cv_grey, threshold_val, max_value, cv2.THRESH_BINARY_INV)
#add 10 pixel border, black
img_cv_border = cv2.copyMakeBorder(img_cv_grey, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=[0, 0, 0])

topil(img_cv_border)

## Erode / Dilate

Reduces / Expands white regions on the image, respectively.

By applying complementary erode/dilate operations you can get rid of gaps and concave parts of an image.

Doc: https://docs.opencv.org/4.x/db/df6/tutorial_erosion_dilatation.html

In [ ]:
# this sets up the shape and size of the erosion filter
eksize = 2
ekernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (2 * eksize + 1, 2 * eksize + 1), (eksize, eksize))

dksize = 5
dkernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (2 * dksize + 1, 2 * dksize + 1), (dksize, dksize))

eroded_cv = cv2.erode(img_cv_border, ekernel)
dilated_cv = cv2.dilate(img_cv_border, dkernel)

display(topil(dilated_cv))

## Extracting Outline

The function for this is called `findContours()`.

Docs: https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html

In [ ]:
colors = [(220,0,0),(0,220,0),(0,0,220),(220,220,0),(0,220,220),(220,0,220)]

draw_cv = cv2.cvtColor(dilated_cv.copy(), cv2.COLOR_GRAY2RGB)

contours, hierarchy = cv2.findContours(image=dilated_cv, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

if contours:
  for idx,con in enumerate(contours):
    cv2.drawContours(draw_cv, [con], 0, colors[idx%len(colors)], 2)

display(PImage.fromarray(draw_cv))

We find the biggest contour from the dilated image, which usually is the sillhouette of the shape. This contour is a set of points which we can use as features for our model.

## Contour objects

are just lists of x,y coordinates, but for some reason opencv adds a mysterious dimension to them.

Instead of this:

```python
[
  [x0,y0], [x1,y1], [x2,y2], ...
]
```

We get this (double array around points):
```python
[
  [[x0,y0]], [[x1,y1]], [[x2,y2]], ...
]
```

We can fix it by using the `squeeze()` function which gets rid of superfluous dimensions in arrays:

In [ ]:
# if we want the first contour, this will turn it into a plain list of (x,y) coordinates
print(len(contours[0].squeeze().tolist()))
print("Contour Area:", cv2.contourArea(contours[0]))
contours[0].squeeze().tolist()


contouring is just grabbing the outer box of the kamon, not the kamon itself. area is calculating the contour around the edge of the screen.

## Filtering

Contours: 
- with points touching the edges of the image
- larger than 70% of the image area
- smaller than 15% of the image area

are not valid.


In [ ]:
def contour_is_valid(c, h, w, m=1):
  for p in c:
    x, y = p[0]
    if x < m or x > w - m - 1 or y < m or y > h - m - 1:
      return False
  return (cv2.contourArea(c) < 0.70 * h * w) and (cv2.contourArea(c) > 0.15 * h * w)

contour_is_valid(contours[0], img.size[1] + 20 , img.size[0] + 20 )

come up with heuristic for my images - what is valid?
- contour that finds the sillhouette

## Further:
Now repeat this process for every image in the random_subset. Images that can be processed and pass the heuristic have their filename, largest contour, and area saved to a dataframe for further work.

In [ ]:
colors = [(220,0,0),(0,220,0),(0,0,220),(220,220,0),(0,220,220),(220,0,220)] # for drawing contour

def contour_from_image(path):
    # process image to get contours
    img = PImage.open(path)
    img.thumbnail((480, 480))
    img_cv = tocv(img)
    if len(img_cv.shape) == 3 and img_cv.shape[2] == 3:  # If the image has 3 channels (RGB)
        img_cv_grey = cv2.cvtColor(img_cv, cv2.COLOR_RGB2GRAY)
    else:  # If the image is already grayscale
        img_cv_grey = img_cv
        #images that are already greyscale have low contrast
        # so we equalize the intesity of each pixel
        img_cv_grey = cv2.equalizeHist(img_cv_grey) 

    #threshold image to binary
    max_value = 255
    threshold_val = 128
    _, img_cv_grey = cv2.threshold(img_cv_grey, threshold_val, max_value, cv2.THRESH_BINARY_INV)
    #add 10 pixel border, black
    img_cv_border = cv2.copyMakeBorder(img_cv_grey, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    #Dilate
    dksize = 8
    dkernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (2 * dksize + 1, 2 * dksize + 1), (dksize, dksize))
    dilated_cv = cv2.dilate(img_cv_border, dkernel)

    draw_cv = cv2.cvtColor(dilated_cv.copy(), cv2.COLOR_GRAY2RGB)

    contours, hierarchy = cv2.findContours(image=dilated_cv, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

    if contours:
        for idx, con in enumerate(contours):
            cv2.drawContours(draw_cv, [con], 0, colors[idx % len(colors)], 2)

        display(PImage.fromarray(draw_cv))

        # check for valid biggest contour
        if contour_is_valid(contours[0], img.size[1] + 20, img.size[0] + 20):
            print("Valid Contour Found")
            print("Contour Area:", cv2.contourArea(contours[0]))
            
            # output file name, path, contour, contour area
            return {
                "file_name": os.path.basename(path),
                "file_path": path,
                "contour": contours[0].squeeze().tolist(),
                "contour_area": cv2.contourArea(contours[0])
            }
        else:
            print("No Valid Contour Found for",path)
    
    # Explicitly return None if no valid contour is found
    return None
        
    



In [ ]:
contour_from_image("random_subset/plant_nadeshiko_1.jpg")
contour_from_image("random_subset/plant_kiri_113.jpg")

Now, do this for all samples in the random_subset. For each sample,
 - save name of sample
 - save file path
 - save largest contour
 - save total area

In [ ]:
#go through all samples in random_subset and save contour data

success_rate = []

random_subset_contours = []
for file_path in os.listdir("random_subset"):
    #run contour extraction
    contour_data = contour_from_image(os.path.join("random_subset", file_path))
    #add to list if contour data is valid
    if contour_data is not None:
        random_subset_contours.append(contour_data)
        success_rate.append(1)
    else:
        success_rate.append(0)

    
    

In [225]:

sum(success_rate)/len(success_rate)
#62% success rate

0.6240740740740741

In [227]:
#turn list into dataframe
random_subset_contours_df = pd.DataFrame(random_subset_contours)
random_subset_contours_df.head()

,file_name,file_path,contour,contour_area
0,geometry_turtle_shell_49.jpg,random_subset/geometry_turtle_shell_49.jpg,"[[64, 2], [63, 3], [62, 4], [61, 4], [60, 5], ...",8562.0
1,object_anchor_10.jpg,random_subset/object_anchor_10.jpg,"[[41, 2], [40, 3], [39, 3], [38, 4], [37, 4], ...",10102.5
2,object_mari_scissors_3.jpg,random_subset/object_mari_scissors_3.jpg,"[[63, 2], [63, 3], [63, 4], [63, 5], [63, 6], ...",8116.0
3,plant_tachibana_5.jpg,random_subset/plant_tachibana_5.jpg,"[[63, 2], [62, 3], [61, 3], [60, 4], [59, 4], ...",9798.5
4,object_kamashiki_4.jpg,random_subset/object_kamashiki_4.jpg,"[[59, 2], [58, 3], [57, 3], [56, 3], [55, 4], ...",12607.0


In [234]:
#give each point in contour a column. If there are less points than max, fill with 0
max_points = max([len(c) for c in random_subset_contours_df['contour']])
max_points
# Create all x and y columns at once
x_columns = pd.DataFrame(
    {f'x_{i}': random_subset_contours_df['contour'].apply(lambda c: c[i][0] if i < len(c) else 0)
     for i in range(max_points)}
)

y_columns = pd.DataFrame(
    {f'y_{i}': random_subset_contours_df['contour'].apply(lambda c: c[i][1] if i < len(c) else 0)
     for i in range(max_points)}
)

# Concatenate the new columns to the original DataFrame
random_subset_contours_df = pd.concat([random_subset_contours_df, x_columns, y_columns], axis=1)

In [235]:
random_subset_contours_df.head()

,file_name,file_path,contour,contour_area,x_0,y_0,x_1,y_1,x_2,y_2,...,y_773,y_774,y_775,y_776,y_777,y_778,y_779,y_780,y_781,y_782
0,geometry_turtle_shell_49.jpg,random_subset/geometry_turtle_shell_49.jpg,"[[64, 2], [63, 3], [62, 4], [61, 4], [60, 5], ...",8562.0,64,2,63,3,62,4,...,0,0,0,0,0,0,0,0,0,0
1,object_anchor_10.jpg,random_subset/object_anchor_10.jpg,"[[41, 2], [40, 3], [39, 3], [38, 4], [37, 4], ...",10102.5,41,2,40,3,39,3,...,0,0,0,0,0,0,0,0,0,0
2,object_mari_scissors_3.jpg,random_subset/object_mari_scissors_3.jpg,"[[63, 2], [63, 3], [63, 4], [63, 5], [63, 6], ...",8116.0,63,2,63,3,63,4,...,0,0,0,0,0,0,0,0,0,0
3,plant_tachibana_5.jpg,random_subset/plant_tachibana_5.jpg,"[[63, 2], [62, 3], [61, 3], [60, 4], [59, 4], ...",9798.5,63,2,62,3,61,3,...,0,0,0,0,0,0,0,0,0,0
4,object_kamashiki_4.jpg,random_subset/object_kamashiki_4.jpg,"[[59, 2], [58, 3], [57, 3], [56, 3], [55, 4], ...",12607.0,59,2,58,3,57,3,...,0,0,0,0,0,0,0,0,0,0


This leaves me with a dataframe with all succesful Kamons, their contours, contour areas, and the x-y coordinates all in their own columns. I can export this dataframe to be further processed before clustering.

In [236]:
#export dataframe to csv
random_subset_contours_df.to_csv("random_subset_contours.csv", index=False)